In [441]:
import os
import glob
import zlib
import pathlib
import tempfile
import subprocess
import itertools
import random
import math
import scipy as sp
import scipy.optimize
import cma
import skimage.draw
from scipy.optimize import differential_evolution, minimize, basinhopping
from sklearn.metrics import pairwise_distances
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [442]:
input_glob = os.path.join("..", "resources", "islands", "*.npy")

In [443]:
input_file = os.path.join("..", "resources", "islands", "moderate_l_01.npy")
input_data = np.load(input_file)

In [444]:
# flip axes
input_data = input_data.T

In [445]:
px.imshow(input_data)

In [446]:
# num_sources = 24
radius = 20
source_size = 4
e = 1e-8

In [447]:
img = np.zeros((40, 40))
img[skimage.draw.disk(
    center=(19.5, 19.5),
    radius=20
)] = 1
px.imshow(img)

In [448]:
townhall_coverage = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0 ],
    [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0 ],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0 ],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0 ],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0 ],
    [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0 ],
    [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
])
townhall_disk = np.nonzero(townhall_coverage)
px.imshow(townhall_coverage)

In [449]:
# num_variables = 4 + input_data.ndim * num_sources
# bounds_lower = [0 for _ in range(num_variables)]
# bounds_upper = [1, 2 * np.pi, 1, 1] + list(input_data.shape) * num_sources
# bounds = list(zip(bounds_lower, bounds_upper))
# bounds

In [450]:
# remove surrounding ocean
interior = np.nonzero(input_data)
input_data = input_data[
    interior[0].min():interior[0].max()+1,
    interior[1].min():interior[1].max()+1,
]

In [451]:
def distance_from_set(s) -> np.ndarray:
    distance = np.where(s, 0.0, np.inf)
    visited = e < s # set grows outward
    for k in itertools.count(1):
        # expand to 3x3 neighbourhood
        visited = e < scipy.ndimage.uniform_filter(np.where(visited, 1.0, 0.0), 3)
        update = (distance == np.inf) & visited
        if not update.any():
            break
        distance[update] = k
    return distance

def signed_distance_from_boundary(s) -> np.ndarray:
    return distance_from_set(s) - np.maximum(0, -1 + distance_from_set(~s))

In [452]:
input_center = (input_data.shape[0] / 2, input_data.shape[1] / 2)
distance_grid = distance_from_set(input_data)
distance_max = distance_grid.max()
px.imshow(distance_grid)

In [453]:
def vector_to_sources(x, grid_mode):

    # sources = x[4:].reshape((-1, 2))
    # center = np.array(input_center)
    # def affine_transform(x):
    #     return center + rotation @ (x - center) + translation


    # sources = 
    if grid_mode:

        phi = x[0]
        rotation = np.array([[np.cos(phi), np.sin(phi)], [-np.sin(phi), np.cos(phi)]])
        translation = x[1:3]

        sources = []
        for u in range(-10, 10):
            for v in range(-10, 10):
                coords = np.array((2 * u + (v % 2), np.sqrt(3) * v))
                # coords -= center
                # coords *= 1 + .5 * scale / radius
                # coords += center
                # coords = affine_transform(coords)
                coords = rotation @ coords + translation
                coords = radius * coords
                coords = coords.astype(int)
                # if ((0 <= coords) & (coords < input_data.shape)).all():
                #     if input_data[coords[0], coords[1]]:

                footprint = skimage.draw.rectangle(
                    start=coords - source_size / 2,
                    extent=(source_size, source_size),
                    shape=input_data.shape,
                )
                if footprint[0].size and input_data[footprint].all():
                    sources.append(coords)
    else: 
        sources = x.reshape((-1, 2))
        sources = sources[0, :] + sources[1:, :]

    return np.array(sources)

    # sources_transformed = list(map(affine_transform, sources))
    # return np.array(sources_transformed)

def sources_to_grid(sources):
    a = np.zeros_like(input_data, dtype=int)
    for s in sources:
        d = skimage.draw.disk(center=np.round(s) - .5, radius=radius, shape=a.shape)
        a[d] += 1
    return a

def constraints(sources):
    bounds = np.array(input_data.shape) - 1

    for s in sources:

        # ensure coordinates are in bounds
        yield abs(s - np.clip(s, 0, bounds)).sum()

        # ensure building is buildable
        footprint = skimage.draw.rectangle(
            start=np.array(s) - source_size / 2,
            extent=(source_size, source_size),
            shape=input_data.shape,
        )
        samples = distance_grid[footprint]
        if samples.size == 0:
            samples = distance_grid
        yield max(0, samples.max(initial=0)) 

    # check lower-triangle-entries of distance matrix
    if sources.size:
        distances = pairwise_distances(np.round(sources))
        distances_tril = distances[np.tril_indices_from(distances, -1)]
        distance_constraints = np.maximum(0, 2 * radius -  distances_tril)

        # increase values in (e, 1) to 1 to prevent underconstraining
        distance_constraints = np.where(e < distance_constraints, np.maximum(1, distance_constraints), distance_constraints)

        yield distance_constraints.sum()
    else:
        yield 0

constraint_weight = 1
def objective_fn(x, grid_mode):
    sources = vector_to_sources(x, grid_mode=grid_mode)
    coverage = sources_to_grid(sources)
    reward_coverage = (input_data * (coverage > 0)).sum()
    value = reward_coverage
    if not grid_mode:
        value -= constraint_weight * sum(constraints(sources))
    return -value

In [454]:
def vector_to_image(x, grid_mode):
    sources = vector_to_sources(x, grid_mode)
    output_shape = input_data.shape[0], input_data.shape[1], 3
    img = np.zeros(output_shape, dtype=np.uint8)
    coverage = sources_to_grid(sources)
    img[:, :, 0] += (255 * (coverage == 0) * (input_data == 1)).astype(np.uint8)
    img[:, :, 1] += (255 * (coverage == 0) * (input_data == 1)).astype(np.uint8)
    img[:, :, 0] += (255 * (coverage > 1) * (input_data == 1)).astype(np.uint8)
    img[:, :, 1] += (255 * (coverage == 1) * (input_data == 1)).astype(np.uint8)
    for s in sources:
        r = skimage.draw.rectangle(
            start=np.array(s) - source_size / 2,
            extent=(source_size, source_size),
            shape=input_data.shape,
        )
        img[r[0], r[1], :] = 255
    return img

In [455]:

iteration = 0
images = []
image_hashes = set()
def cb(x, grid_mode, *args):
    global iteration
    iteration += 1
    img = vector_to_image(x, grid_mode)
    # image_hash = hash(img.data.tobytes())
    # if image_hash in image_hashes:
    #     return
    # image_hashes.add(image_hash)
    img = np.repeat(img, 2, 0)
    img = np.repeat(img, 2, 1)
    img = np.pad(img, ((100, 0), (0, 0), (0, 0)))
    image = Image.fromarray(img)
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("arial.ttf", 16)
    draw.text(
        (0, 0),
        f"Iteration : {iteration}",
        font=font,
    )

    
    sources = vector_to_sources(x, grid_mode)
    coverage = sources_to_grid(sources)
    coverage_tiles = ((coverage > 0) & input_data).sum()
    draw.text(
        (0, 20),
        f"Tiles covered : {coverage_tiles}",
        font=font,
    )
    constraint_loss = sum(constraints(sources))
    draw.text(
        (0, 40),
        f"Constraint loss : {round(constraint_loss, 3)}",
        font=font,
    )

    images.append(image)
    for fname in f"iterations/{iteration}.png", "latest.png":
        output_path = os.path.join("..", "resources", "optimization", fname)
        image.save(output_path)

In [456]:
x_tolsimir = np.array([
    0, 0, 0, 0,
    34, 76,
    13, 111,
    75, 78,
    53, 112,
    31, 147,
    71, 149,
    89, 186,
    111, 152,
    94, 115,
    130, 188,
    170, 190,
    152, 153,
    193, 157,
    233, 163,
    136, 116,
    177, 120,
    218, 125,
    116, 81,
    98, 43,
    121, 9,
    142, 44,
    182, 49,
    158, 82,
    200, 86,
])

px.imshow(vector_to_image(x_tolsimir.flatten(), grid_mode=False))

In [457]:
# objective_fn(x_tolsimir, grid_mode=False), sum(constraints(x_tolsimir))

In [458]:
# options = cma.CMAOptions()
# options.set("bounds", [bounds_lower, bounds_upper])
# options.set("seed", 42)
# # options.set("popsize", 500)
# # options.set("tolflatfitness", 32)
# # options.set("tolfun", 0)
# # options.set("tolfunhist", 0)
# options.set("integer_variables", list(range(4, num_variables)))

In [459]:

# res = differential_evolution(
#     lambda x: -reward_fn(x),
#     bounds=bounds,
#     callback=cb,
#     constraints=scipy.optimize.NonlinearConstraint(lambda x: sum(constraints(x)), -np.inf, 1e-2),
#     disp=True,
#     integrality=True,
#     maxiter=10_000,
#     # tol=1e-8,
#     # strategy="randtobest1bin",
#     # popsize=32,
#     # workers=2,
#     # atol=1.0,
#     # tol=1e-5,
#     # recombination=.9,
# )

In [460]:
cma.CMAOptions()

{'AdaptSigma': 'True  # or False or any CMAAdaptSigmaBase class e.g. CMAAdaptSigmaTPA, CMAAdaptSigmaCSA',
 'CMA_active': 'True  # negative update, conducted after the original update',
 'CMA_active_injected': '0  #v weight multiplier for negative weights of injected solutions',
 'CMA_cmean': '1  # learning rate for the mean value',
 'CMA_const_trace': 'False  # normalize trace, 1, True, "arithm", "geom", "aeig", "geig" are valid',
 'CMA_diagonal': '0*100*N/popsize**0.5  # nb of iterations with diagonal covariance matrix, True for always',
 'CMA_diagonal_decoding': '0  # learning rate multiplier for additional diagonal update',
 'CMA_eigenmethod': 'np.linalg.eigh  # or cma.utilities.math.eig or pygsl.eigen.eigenvectors',
 'CMA_elitist': 'False  #v or "initial" or True, elitism likely impairs global search performance',
 'CMA_injections_threshold_keep_len': '1  #v keep length if Mahalanobis length is below the given relative threshold',
 'CMA_mirrors': 'popsize < 6  # values <0.5 are int

In [461]:



# res = scipy.optimize.dual_annealing(
#     loss_fn,
#     bounds=bounds,
#     callback=cb,
#     maxiter=10_000,
# )


# x0 = np.array([ 48.86787419, 145.05459966, 150.06288545,  47.92156405,
#        138.99561885,  87.28520437,  71.10871424,  68.33506996,
#         21.76786652, 115.05052127, 148.73939368, 194.0049809 ,
#        190.24731688,  38.854061  , 110.31125759,  59.09129294,
#        196.97066102, 144.4597769 , 178.05749153,  77.22961749,
#        128.83280234, 126.21920866, 224.49979488, 173.93801628,
#        157.65373721, 155.09534156, 235.95264006, 134.9194016 ,
#        118.57426577, 164.97182244,  61.08120205, 107.14704587,
#         31.14791093,  76.45051155,  88.7839089 , 137.4468036 ,
#        100.15640686,  98.22663478, 206.91228001, 105.07770597,
#          8.54136674, 152.82120424, 121.48876576,  19.50708521,
#        167.73447415, 116.44081604,  81.68628624, 180.17291539])
# sigma=1.0

# x0=[(li + ui) / 2 for li, ui in bounds]
# sigma0=math.sqrt((input_data.shape[0] * input_data.shape[1]) / 12)

seed = 43

options = cma.CMAOptions()
options.set("bounds", [[-np.pi, -1, -1], [+np.pi, 1, 1]])
options.set("seed", seed)
options.set("popsize", 64)
# options.set("tolflatfitness", 32)
# options.set("tolfun", 0)
# options.set("tolfunhist", 0)
# options.set("integer_variables", list(range(3)))

es = cma.CMAEvolutionStrategy([0, 0, 0], 1.0, options)

while not es.stop():
# for _ in range(20):
    solutions = es.ask()
    values = [objective_fn(x, grid_mode=True) for x in solutions]
    es.tell(solutions, values)
    es.disp()
    cb(es.best.x, grid_mode=True)

    
sources = vector_to_sources(es.best.x, grid_mode=True)
num_variables = input_data.ndim * len(sources)
bounds_lower = 2 * [-1e5] + [0 for _ in range(num_variables)]
bounds_upper = 2 * [1e5] + list(input_data.shape) * len(sources)

x1 = np.concatenate(([0, 0], sources.flatten()))


options = cma.CMAOptions()
options.set("bounds", [bounds_lower, bounds_upper])
options.set("seed", seed)
# options.set("maxiter", 100)
options.set("popsize", 256)
# options.set("tolflatfitness", 32)
# options.set("tolfun", 0)
# options.set("tolfunhist", 0)
options.set("integer_variables", list(range(2, len(x1))))

# sigma1 = 1e-1

constraint_weight = 32
while True:

    es = cma.CMAEvolutionStrategy(x1, 0.3, options)
    # for _ in range(100):
    while not es.stop():
        solutions = es.ask()
        values = [objective_fn(x, grid_mode=False) for x in solutions]
        es.tell(solutions, values)
        es.disp()
        cb(es.best.x, grid_mode=False)

    constraint_loss = sum(constraints(vector_to_sources(es.best.x, grid_mode=False)))
    if 0 == constraint_loss:
        break
    
    x1 = es.best.x
    constraint_weight *= 2
    # sigma1 = es.sigma0
    # sigma1 = 1e-1 * es.sigma0

# res = cma.fmin2(
#     objective_function=lambda x: -reward_fn(x),
#     x0=x0,
#     sigma0=sigma0,
#     # constraints=lambda x: list(constraints(x)),
#     # find_feasible_first=True,
#     # find_feasible_final=True,
#     restarts=5,
#     restart_from_best=True,
#     callback=lambda es: cb(es.result.xbest),
#     options=options,
# )

# res

c:\Users\volke\PycharmProjects\anno-tools\.venv\Lib\site-packages\cma\evolution_strategy.py:1312: UserWarning:

Sampling standard deviation i=1 (and 1 others) at iteration 0 multiplied by 0.6666555556481476 to stds[1]=0.6666666666666666



(32_w,64)-aCMA-ES (mu_w=17.6,w_1=11%) in dimension 3 (seed=43, Fri Oct  3 02:50:15 2025)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     64 -2.636500000000000e+04 1.0e+00 9.84e-01  5e-01  1e+00 0:00.9
    2    128 -2.665300000000000e+04 2.0e+00 1.22e+00  6e-01  2e+00 0:01.7
    3    192 -2.614700000000000e+04 2.2e+00 1.06e+00  5e-01  1e+00 0:02.6
    7    448 -2.569100000000000e+04 5.5e+00 1.08e+00  3e-01  2e+00 0:06.1
   12    768 -2.618300000000000e+04 2.9e+00 1.27e+00  3e-01  1e+00 0:10.5
   18   1152 -2.607500000000000e+04 4.6e+00 1.51e+00  3e-01  2e+00 0:15.7
   25   1600 -2.679500000000000e+04 6.0e+00 1.37e+00  2e-01  1e+00 0:21.8
   34   2176 -2.554900000000000e+04 6.6e+00 1.99e+00  2e-01  8e-01 0:29.6
   44   2816 -2.580800000000000e+04 3.4e+00 1.72e+00  7e-02  1e-01 0:38.3
   55   3520 -2.630200000000000e+04 1.0e+01 1.88e+00  5e-02  8e-02 0:47.8
   67   4288 -2.634200000000000e+04 2.6e+01 1.80e+00  2e-03  6e-03 0:58.7
   76   4864 -2.634200000

In [462]:

# res = scipy.optimize.dual_annealing(
#     lambda x: -reward_fn(x),
#     bounds=bounds,
#     callback=cb,
#     maxiter=10_000,
#     x0=es.best.x,
# )
# res.x

In [463]:
animation_path = os.path.join("..", "resources", "optimization", "animation.gif")
images[0].save(animation_path, save_all=True, append_images=images[1:], duration=1, loop=0)